In [18]:
import os
import flopy as f
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
ws = "work/ex11"
name = "modflow_project"
exe_name = "../mf6.exe"

In [20]:
from shutil import which
print(which(os.path.abspath(exe_name)))

None


In [21]:
os.path.abspath(exe_name)

'/Users/omowumierukubami/Desktop/notebooks/mf6.exe'

In [22]:
os.path.isfile(exe_name)

True

In [23]:
# create simulation
sim = f.mf6.MFSimulation(sim_name=name, sim_ws=ws, exe_name=exe_name)

In [24]:
# Temporal discretization
nper = 1
perlen = 8640000
nstp = 1
tdis = f.mf6.ModflowTdis(sim, time_units="seconds", nper = 1, perioddata=[(0, 8640000, 1)])

In [25]:
ims = f.mf6.ModflowIms(sim)

In [26]:
# create model object
gwf = f.mf6.ModflowGwf(sim, modelname=name)

In [27]:
# Spatial Discretization
nlay, nrow, ncol = 1, 25, 20
delr=delc=50
top=120
botm=75
ibound = np.ones((nlay, nrow, ncol), dtype=np.int64)
ibound[:,:,:7]=0
ibound[:,:8,11]=0

Dis = f.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                            botm=botm,delr=delr, delc=delc, idomain=ibound)

In [28]:
# Node Properties Definition
kh=kva=10e-5
laytyp=[1]

npf = f.mf6.ModflowGwfnpf(gwf,save_specific_discharge=True, icelltype=laytyp, k=kh, k33=kva)

In [29]:
# Defining initial conditions

start=80 * np.ones((nlay,nrow,ncol), dtype=np.float64)
ic = f.mf6.ModflowGwfic(gwf, strt=start)

In [30]:
# Recharge package
rch = f.mf6.ModflowGwfrcha(gwf, recharge=2e-8)

In [31]:
# Defining Constant Head
chd_rec= {0:[[0,1,15,100.0,100.0]]}
chd=f.mf6.ModflowGwfchd(gwf, stress_period_data=chd_rec)


In [32]:
headfile=f"{name}.hds"
head_filerecord=[headfile]
budgetfile=f"{name}.cbb"
budget_filerecord=[budgetfile]
saverecord=[("HEAD","ALL"),("BUDGET","ALL")]
printrecord=[("HEAD","LAST")]

oc=f.mf6.ModflowGwfoc(gwf,saverecord=saverecord,head_filerecord=head_filerecord,budget_filerecord=budget_filerecord,printrecord=printrecord)

In [33]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package ims_-1...
  writing model modflow_project...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package rcha_0...
    writing package chd_0...
INFORMATION: maxbound in ('gwf6', 'chd', 'dimensions') changed to 1 based on size of stress_period_data
    writing package oc...


In [34]:
success, buff = sim.run_simulation()
assert success, "MODFLOW 6 did not terminate normally"

Exception: The program ../mf6.exe does not exist or is not executable.